In [2]:
#Import packages
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import GRB
import matplotlib.pyplot as plt
import docx

from datetime import date

today = date.today()

from results_output import output_to_table

import warnings # ignore warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

In [3]:

# change name of path 
food_data = pd.read_csv("../data/cleaned_taco_bell_menu_items_2024-08-22.csv", index_col=0)
food_data.reset_index(inplace=True)
print(food_data.shape)
food_data

(63, 18)


,index,item_name,price,menu_section,Calories,Total Fat (g),Saturated Fat (g),Trans Fat (g),Cholesterol (mg),Sodium (mg),Total Carbohydrates (g),Dietary Fiber (g),Protein (g),Vitamin D (mcg),Calcium (mg),Iron (mg),Potassium (mg),Total Sugars (g)
0,0,Steak Cheesy Street Chalupas,5.49,Tacos,410.0,25.0,6.0,0.0,35.0,750.0,27.0,3.0,19.0,0.0,150.0,3.6,210.0,4.0
1,1,Cantina Chicken Cheesy Street Chalupas,5.49,Tacos,410.0,25.0,6.0,0.0,40.0,670.0,28.0,4.0,19.0,0.0,140.0,3.7,270.0,3.0
2,2,Double Stacked Taco,1.99,Tacos,320.0,16.0,5.0,0.0,25.0,600.0,34.0,4.0,11.0,0.0,130.0,1.9,300.0,3.0
3,3,Soft Taco,1.89,Tacos,180.0,8.0,4.0,0.0,25.0,490.0,18.0,3.0,9.0,0.0,100.0,1.7,130.0,2.0
4,4,Soft Taco Supreme®,2.89,Tacos,210.0,10.0,5.0,0.0,30.0,510.0,20.0,3.0,9.0,0.0,120.0,1.7,200.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,92,Breakfast Crunchwrap Bacon,4.29,Breakfast,670.0,40.0,13.0,0.0,140.0,1300.0,52.0,4.0,21.0,0.0,330.0,3.1,390.0,5.0
59,93,Breakfast Crunchwrap Sausage,4.29,Breakfast,750.0,49.0,17.0,0.0,150.0,1210.0,53.0,4.0,21.0,0.0,340.0,3.3,400.0,5.0
60,94,Hash Brown,1.79,Breakfast,160.0,11.0,1.0,0.0,0.0,280.0,14.0,1.0,1.0,0.0,10.0,0.0,190.0,0.0
61,95,Cinnabon Delights® 2 Pack,2.29,Breakfast,170.0,11.0,3.5,0.0,5.0,70.0,15.0,1.0,2.0,0.0,10.0,0.0,30.0,18.0


In [4]:
#Defining variables
price = food_data['price'].tolist()
menu_item = food_data['item_name'].tolist()
section_name = food_data['menu_section'].tolist()

calories = food_data['Calories'].tolist()
protein = food_data['Protein (g)'].tolist()
totalCarbohydratets = food_data['Total Carbohydrates (g)'].tolist()
dietaryFiber = food_data['Dietary Fiber (g)'].tolist()
totfat = food_data['Total Fat (g)'].tolist()
statFat = food_data['Saturated Fat (g)'].tolist()
transFat = food_data['Trans Fat (g)'].tolist()
cholesterol = food_data['Cholesterol (mg)'].tolist()
sodium = food_data['Sodium (mg)'].tolist()
sugars = food_data['Total Sugars (g)'].tolist()

vitamin_d = food_data['Vitamin D (mcg)'].tolist()
Calcium = food_data['Calcium (mg)'].tolist()
iron = food_data['Iron (mg)'].tolist()
Potassium = food_data['Potassium (mg)'].tolist()


## RQ: Can Taco Bell's menu provide a balanced diet that aligns with affordability and nutritional requirements, especially for individuals with limited financial means or access to diverse food sources?

### Sets and Indices
Let $F$ denote the set of food items considered on the menu: $i \in F$

Let $N$ denote the set of nutritional requirements: $j \in N$.

Let $S$ denote the set of menu sections on the menu: $z \in S$

### Data
$n_{i,j}$ = How much of nutrient $j$ one menu item $i$ has, $i \in F, j \in N$.

$c_i$ = The price of one serving of menu item $i$, $i \in F$ (per item cost)

$m_j$ = Minimum daily requirements of nutrient $j$, $j
\in N$

## Model 1: Minimizing costs
### Decision Variable
* $x_i$: The number of menu items $i$ to include in the diet; $x_1,..,n$


### Objective
* Minimize costs $\sum_{i \in F} c_i * x_i$.


### Nutrional Constraints
* $ \sum_{i \in F} n_{i,j} * x_i \geq m_j,  j \in N$


### Non-Negativity Constraint
* $x_i \geq 0, i=1,\ldots,n$ (non-negativity constraint)


In [5]:
m_male1_LP = gp.Model("TBELL_MODEL: EOM-LP")
m_male1_LP.Params.LogToConsole = 1 # Noisy output

max_totFat = (2400*0.35)//9
max_statFat = (2400*0.10)//9


foods = [m_male1_LP.addVar(obj=food_data['price'][i], vtype="C", name=food_data['item_name'][i]) for i in range(len(food_data))]


m_male1_LP.setObjective(gp.quicksum(price[i]*foods[i] for i in range(len(food_data))), GRB.MINIMIZE)


m_male1_LP.addConstr(gp.quicksum(calories[i]*foods[i] for i in range(len(food_data))) >= 2400/3, name="calories")
m_male1_LP.addConstr(gp.quicksum(protein[i]*foods[i] for i in range(len(food_data))) >= 56/3,name="protein")
m_male1_LP.addConstr(gp.quicksum(totalCarbohydratets[i]*foods[i] for i in range(len(food_data))) >= 130/3, name="totalCarbohydratets")
m_male1_LP.addConstr(gp.quicksum(dietaryFiber[i]*foods[i] for i in range(len(food_data))) >= 34/3, name="dietaryFiber")
m_male1_LP.addConstr(gp.quicksum(totfat[i]*foods[i] for i in range(len(food_data))) >= max_totFat/3, name="totFat")
m_male1_LP.addConstr(gp.quicksum(statFat[i]*foods[i] for i in range(len(food_data))) >= max_statFat/3, name="statFat")
m_male1_LP.addConstr(gp.quicksum(transFat[i]*foods[i] for i in range(len(food_data))) >= 0/3, name="transFat")
m_male1_LP.addConstr(gp.quicksum(cholesterol[i]*foods[i] for i in range(len(food_data))) >= 300/3, name="cholesterol")
m_male1_LP.addConstr(gp.quicksum(sodium[i]*foods[i] for i in range(len(food_data)))>=2300/3, name="sodium")
m_male1_LP.addConstr(gp.quicksum(sugars[i]*foods[i] for i in range(len(food_data)))>=10/3, name="sugars")

# Micro Nutrients
m_male1_LP.addConstr(gp.quicksum(Potassium[i]*foods[i] for i in range(len(food_data))) >= 3400/3, name="Potassium")
m_male1_LP.addConstr(gp.quicksum(iron[i]*foods[i] for i in range(len(food_data))) >= 8/3, name="iron")
m_male1_LP.addConstr(gp.quicksum(Calcium[i]*foods[i] for i in range(len(food_data))) >= 1000/3, name="Calcium")
m_male1_LP.addConstr(gp.quicksum(vitamin_d[i]*foods[i] for i in range(len(food_data))) <= 15/3, name="vitamin_d")


m_male1_LP.ModelSense = 1 #Minimization problem

m_male1_LP.optimize()

Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-06
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M2 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 14 rows, 63 columns and 723 nonzeros
Model fingerprint: 0xdc4527fc
Coefficient statistics:
  Matrix range     [1e-01, 2e+03]
  Objective range  [1e+00, 8e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+00, 1e+03]
Presolve removed 1 rows and 0 columns
Presolve time: 0.00s
Presolved: 13 rows, 63 columns, 721 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.921667e+02   0.000000e+00      0s
       4    4.9209581e+00   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.01 seconds (0.00 work units)
Optimal objective  4.920958084e+00


In [6]:
if m_male1_LP.status == gp.GRB.OPTIMAL:
    print(f'Optimal combination of foods for one meal costs ${round(m_male1_LP.objVal, 2)}:')


    df = output_to_table(m_male1_LP, foods, food_data)

df


Optimal combination of foods for one meal costs $4.92:


,Menu,Servings,Meal Price,Section Name,Nutrient,Values,Recommended Minimum Per Meal (%)
0,Cheesy Bean and Rice Burrito,2.18,4.92,Veggie Cravings,Calories,1232.73,154.09
1,Cheesy Toasted Breakfast Burrito Potato,0.94,,Breakfast,Protein (g),28.00,155.56
2,,,,,Total Carbohydrate (g),161.00,374.42
3,,,,,Dietary Fiber (g),16.00,177.78
4,,,,,Total Fat (g),50.00,161.29
5,,,,,Saturated Fat (g),13.00,162.50
6,,,,,Trans Fat (g),0.00,0.00
7,,,,,Cholesterol (mg),100.00,100.00
8,,,,,Sodium (mg),2767.00,361.23
9,,,,,Sugars (g),15.00,500.00


## Model 2: Minimizing Cost, Integer Programming
### Decision Variable
* $x_i$: The number of menu items $i$ to include in the diet; $x_1,..,n$


### Objective
* Minimize costs $\sum_{i \in F} c_i * x_i$.


### Nutrional Constraints
* $ \sum_{i \in F} n_{i,j} * x_i \geq m_j,  j \in N$


### Non-Negativity Constraint
* $x_i \geq 0, i=1,\ldots,n$ (non-negativity, and integer constraint)


In [7]:
m_male2_IP = gp.Model("TBELL_MODEL: EOM-IP")
m_male2_IP.Params.LogToConsole = 1 # Noisy output

max_totFat = (2400*0.35)//9
max_statFat = (2400*0.10)//9


foods = [m_male2_IP.addVar(obj=food_data['price'][i], vtype="I", name=food_data['item_name'][i]) for i in range(len(food_data))]


m_male2_IP.setObjective(gp.quicksum(price[i]*foods[i] for i in range(len(food_data))), GRB.MINIMIZE)


m_male2_IP.addConstr(gp.quicksum(calories[i]*foods[i] for i in range(len(food_data))) >= 2400/3, name="calories")

m_male2_IP.addConstr(gp.quicksum(protein[i]*foods[i] for i in range(len(food_data))) >= 56/3,name="protein")
m_male2_IP.addConstr(gp.quicksum(totalCarbohydratets[i]*foods[i] for i in range(len(food_data))) >= 130/3, name="totalCarbohydratets")
m_male2_IP.addConstr(gp.quicksum(dietaryFiber[i]*foods[i] for i in range(len(food_data))) >= 34/3, name="dietaryFiber")

m_male2_IP.addConstr(gp.quicksum(totfat[i]*foods[i] for i in range(len(food_data))) >= max_totFat/3, name="totFat")

m_male2_IP.addConstr(gp.quicksum(statFat[i]*foods[i] for i in range(len(food_data))) >= max_statFat/3, name="statFat")
m_male2_IP.addConstr(gp.quicksum(transFat[i]*foods[i] for i in range(len(food_data))) >= 0/3, name="transFat")
m_male2_IP.addConstr(gp.quicksum(cholesterol[i]*foods[i] for i in range(len(food_data))) >= 300/3, name="cholesterol")

m_male2_IP.addConstr(gp.quicksum(sodium[i]*foods[i] for i in range(len(food_data)))>=2300/3, name="sodium")

m_male2_IP.addConstr(gp.quicksum(sugars[i]*foods[i] for i in range(len(food_data)))>=10/3, name="sugars")

# Micro Nutrients
m_male2_IP.addConstr(gp.quicksum(Potassium[i]*foods[i] for i in range(len(food_data))) >= 3400/3, name="Potassium")
m_male2_IP.addConstr(gp.quicksum(iron[i]*foods[i] for i in range(len(food_data))) >= 8/3, name="iron")
m_male2_IP.addConstr(gp.quicksum(Calcium[i]*foods[i] for i in range(len(food_data))) >= 1000/3, name="Calcium")



m_male2_IP.ModelSense = 1 #Minimization problem

m_male2_IP.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M2 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 13 rows, 63 columns and 713 nonzeros
Model fingerprint: 0x7af412ec
Variable types: 0 continuous, 63 integer (0 binary)
Coefficient statistics:
  Matrix range     [2e-01, 2e+03]
  Objective range  [1e+00, 8e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+00, 1e+03]
Found heuristic solution: objective 39.2400000
Presolve removed 4 rows and 33 columns
Presolve time: 0.00s
Presolved: 9 rows, 30 columns, 268 nonzeros
Found heuristic solution: objective 7.2700000
Variable types: 0 continuous, 30 integer (0 binary)

Root relaxation: objective 4.947425e+00, 3 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    4.94743    0    2    7.27000

In [8]:
if m_male2_IP.status == gp.GRB.OPTIMAL:
    print(f'Optimal combination of foods for one meal costs ${round(m_male2_IP.objVal, 2)}:')


    df2 = output_to_table(m_male2_IP, foods, food_data)

df2


Optimal combination of foods for one meal costs $5.27:


,Menu,Servings,Meal Price,Section Name,Nutrient,Values,Recommended Minimum Per Meal (%)
0,Bean Burrito,1.0,5.27,Veggie Cravings,Calories,1120.0,140.00
1,Cheesy Bean and Rice Burrito,1.0,,Veggie Cravings,Protein (g),31.0,172.22
2,Cheesy Toasted Breakfast Burrito Potato,1.0,,Breakfast,Total Carbohydrate (g),153.0,355.81
3,,,,,Dietary Fiber (g),17.0,188.89
4,,,,,Total Fat (g),41.0,132.26
5,,,,,Saturated Fat (g),12.0,150.00
6,,,,,Trans Fat (g),0.0,0.00
7,,,,,Cholesterol (mg),105.0,105.00
8,,,,,Sodium (mg),2790.0,364.23
9,,,,,Sugars (g),14.0,466.67


## Model 3: Minimizing Cost, Integer Programming
### Decision Variable
* $x_i$: The number of menu items $i$ to include in the diet; $x_1,..,n$


### Objective
* Minimize costs $\sum_{i \in F} c_i * x_i$.


### Nutrional Constraints
* $ \sum_{i \in F} n_{i,j} * x_i \geq m_j,  j \in N$

### Upper bound on items 
* $x_i \leq 1  \forall i \in F$

### Non-Negativity Constraint
* $x_i \geq 0, i=1,\ldots,n$ (non-negativity, and integer constraint)


In [9]:
m_male3_IP = gp.Model("TBELL_MODEL: EOM-IP2")
m_male3_IP.Params.LogToConsole = 1 # Noisy output

max_totFat = (2400*0.35)//9
max_statFat = (2400*0.10)//9


foods = [m_male3_IP.addVar(obj=food_data['price'][i], vtype="I", ub=1, name=food_data['item_name'][i]) for i in range(len(food_data))]


m_male3_IP.setObjective(gp.quicksum(price[i]*foods[i] for i in range(len(food_data))), GRB.MINIMIZE)


m_male3_IP.addConstr(gp.quicksum(calories[i]*foods[i] for i in range(len(food_data))) >= 2400/3, name="calories")
m_male3_IP.addConstr(gp.quicksum(protein[i]*foods[i] for i in range(len(food_data))) >= 56/3,name="protein")
m_male3_IP.addConstr(gp.quicksum(totalCarbohydratets[i]*foods[i] for i in range(len(food_data))) >= 130/3, name="totalCarbohydratets")
m_male3_IP.addConstr(gp.quicksum(dietaryFiber[i]*foods[i] for i in range(len(food_data))) >= 34/3, name="dietaryFiber")
m_male3_IP.addConstr(gp.quicksum(totfat[i]*foods[i] for i in range(len(food_data))) >= max_totFat/3, name="totFat")
m_male3_IP.addConstr(gp.quicksum(statFat[i]*foods[i] for i in range(len(food_data))) >= max_statFat/3, name="statFat")
m_male3_IP.addConstr(gp.quicksum(transFat[i]*foods[i] for i in range(len(food_data))) >= 0/3, name="transFat")
m_male3_IP.addConstr(gp.quicksum(cholesterol[i]*foods[i] for i in range(len(food_data))) >= 300/3, name="cholesterol")
m_male3_IP.addConstr(gp.quicksum(sodium[i]*foods[i] for i in range(len(food_data)))>=2300/3, name="sodium")
m_male3_IP.addConstr(gp.quicksum(sugars[i]*foods[i] for i in range(len(food_data)))>=10/3, name="sugars")

# Micro Nutrients
m_male3_IP.addConstr(gp.quicksum(Potassium[i]*foods[i] for i in range(len(food_data))) >= 3400/3, name="Potassium")
m_male3_IP.addConstr(gp.quicksum(iron[i]*foods[i] for i in range(len(food_data))) >= 8/3, name="iron")
m_male3_IP.addConstr(gp.quicksum(Calcium[i]*foods[i] for i in range(len(food_data))) >= 1000/3, name="Calcium")



m_male3_IP.ModelSense = 1 #Minimization problem

m_male3_IP.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M2 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 13 rows, 63 columns and 713 nonzeros
Model fingerprint: 0x6dd732de
Variable types: 0 continuous, 63 integer (0 binary)
Coefficient statistics:
  Matrix range     [2e-01, 2e+03]
  Objective range  [1e+00, 8e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 1e+03]
Found heuristic solution: objective 17.3600000
Presolve removed 3 rows and 2 columns
Presolve time: 0.00s
Presolved: 10 rows, 61 columns, 583 nonzeros
Variable types: 0 continuous, 61 integer (60 binary)
Found heuristic solution: objective 8.6700000

Root relaxation: objective 5.231324e+00, 3 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    5.23132    0    2    8.6700

In [10]:
if m_male3_IP.status == gp.GRB.OPTIMAL:
    print(f'Optimal combination of foods for one meal costs ${round(m_male3_IP.objVal, 2)}:')


    df3 = output_to_table(m_male3_IP, foods, food_data)

df3


Optimal combination of foods for one meal costs $5.27:


,Menu,Servings,Meal Price,Section Name,Nutrient,Values,Recommended Minimum Per Meal (%)
0,Bean Burrito,1.0,5.27,Veggie Cravings,Calories,1120.0,140.00
1,Cheesy Bean and Rice Burrito,1.0,,Veggie Cravings,Protein (g),31.0,172.22
2,Cheesy Toasted Breakfast Burrito Potato,1.0,,Breakfast,Total Carbohydrate (g),153.0,355.81
3,,,,,Dietary Fiber (g),17.0,188.89
4,,,,,Total Fat (g),41.0,132.26
5,,,,,Saturated Fat (g),12.0,150.00
6,,,,,Trans Fat (g),0.0,0.00
7,,,,,Cholesterol (mg),105.0,105.00
8,,,,,Sodium (mg),2790.0,364.23
9,,,,,Sugars (g),14.0,466.67


## Model 4: Minimizing Cost, Integer Programming
### Decision Variables
* $x_i$: The number of menu items $i$ to include in the diet; $x_1,..,n$
* $y_i=1$ if food item $i$ from menu section $z$ is served, and 0 otherwise: $i \in F, z \in S$
i∈F,z∈S$

### Objective
* Minimize costs $\sum_{i \in F} c_i * x_i$.


### Nutrional Constraints
* $ \sum_{i \in F} n_{i,j} * x_i \geq m_j,  j \in N$

### Upper bound on items 
* $x_i \leq 1  \forall i \in F$

### Section Constraints
* $\sum_{ i \in F, z_{i} = z} x_i \geq 1 * y_z \forall z \in S$
* $\sum_{z \in S} y_z \geq 3$

### Non-Negativity Constraint
* $x_i \geq 0, i=1,\ldots,n$ (non-negativity, and integer constraint)
* $y_z \in {0, 1} \forall z \in S$ (binary)  

In [11]:
m_male4_IP = gp.Model("TBELL_MODEL: EOM-IP3")
m_male4_IP.Params.LogToConsole = 1 # Noisy output

max_totFat = (2400*0.35)//9
max_statFat = (2400*0.10)//9

sections = food_data['menu_section'].unique()

# Create a multi-index for items where the first level index is the section and the second level is the item
items = food_data.set_index(['menu_section', 'item_name'])

foods = [m_male4_IP.addVar(obj=food_data['price'][i], vtype="I", ub=1, name=food_data['item_name'][i]) for i in range(len(food_data))]

section_served = {section: m_male4_IP.addVar(vtype="B", name=f"serve_{section}") for section in sections}


m_male4_IP.setObjective(gp.quicksum(price[i]*foods[i] for i in range(len(food_data))), GRB.MINIMIZE)


#### Constraints
# Macro Nutrients  
m_male4_IP.addConstr(gp.quicksum(calories[i]*foods[i] for i in range(len(food_data))) >= 2400/3, name="calories")

m_male4_IP.addConstr(gp.quicksum(protein[i]*foods[i] for i in range(len(food_data))) >= 56/3,name="protein")
m_male4_IP.addConstr(gp.quicksum(totalCarbohydratets[i]*foods[i] for i in range(len(food_data))) >= 130/3, name="totalCarbohydratets")
m_male4_IP.addConstr(gp.quicksum(dietaryFiber[i]*foods[i] for i in range(len(food_data))) >= 34/3, name="dietaryFiber")

m_male4_IP.addConstr(gp.quicksum(totfat[i]*foods[i] for i in range(len(food_data))) >= max_totFat/3, name="totFat")

m_male4_IP.addConstr(gp.quicksum(statFat[i]*foods[i] for i in range(len(food_data))) >= max_statFat/3, name="statFat")
m_male4_IP.addConstr(gp.quicksum(transFat[i]*foods[i] for i in range(len(food_data))) >= 0/3, name="transFat")
m_male4_IP.addConstr(gp.quicksum(cholesterol[i]*foods[i] for i in range(len(food_data))) >= 300/3, name="cholesterol")

m_male4_IP.addConstr(gp.quicksum(sodium[i]*foods[i] for i in range(len(food_data)))>=2300/3, name="sodium")

m_male4_IP.addConstr(gp.quicksum(sugars[i]*foods[i] for i in range(len(food_data)))>=10/3, name="sugars")

# Micro Nutrients
m_male4_IP.addConstr(gp.quicksum(Potassium[i]*foods[i] for i in range(len(food_data))) >= 3400/3, name="Potassium")
m_male4_IP.addConstr(gp.quicksum(iron[i]*foods[i] for i in range(len(food_data))) >= 8/3, name="iron")
m_male4_IP.addConstr(gp.quicksum(Calcium[i]*foods[i] for i in range(len(food_data))) >= 1000/3, name="Calcium")

# Constraint for each section: if y_i = 1, then at least one item from that section is served
for section in sections:
    m_male4_IP.addConstr(gp.quicksum(foods[i] for i in range(len(food_data)) if section_name[i] == section) >= section_served[section], f"serve_at_least_one_{section}")


# Constraint to ensure a minimum number of sections are served
min_sections_to_served = 3 # Example value for minimum sections to be served
m_male4_IP.addConstr(gp.quicksum(section_served[section] for section in sections) >= min_sections_to_served, "min_sections_served")

m_male4_IP.ModelSense = 1 #Minimization problem

m_male4_IP.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M2 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 23 rows, 72 columns and 794 nonzeros
Model fingerprint: 0xdf6328b0
Variable types: 0 continuous, 72 integer (9 binary)
Coefficient statistics:
  Matrix range     [2e-01, 2e+03]
  Objective range  [1e+00, 8e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 1e+03]
Found heuristic solution: objective 40.0200000
Found heuristic solution: objective 21.8500000
Presolve removed 4 rows and 3 columns
Presolve time: 0.00s
Presolved: 19 rows, 69 columns, 660 nonzeros
Variable types: 0 continuous, 69 integer (68 binary)
Found heuristic solution: objective 16.4600000

Root relaxation: objective 5.574110e+00, 15 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node 

In [12]:
if m_male4_IP.status == gp.GRB.OPTIMAL:
    print(f'Optimal combination of foods for one meal costs ${round(m_male4_IP.objVal, 2)}:')


    df4 = output_to_table(m_male4_IP, foods, food_data)

df4


Optimal combination of foods for one meal costs $6.07:


,Menu,Servings,Meal Price,Section Name,Nutrient,Values,Recommended Minimum Per Meal (%)
0,Loaded Beef Nachos,1.0,6.07,Nachos,Calories,1030.0,128.75
1,Spicy Potato Soft Taco,1.0,,Veggie Cravings,Protein (g),26.0,144.44
2,Cheesy Toasted Breakfast Burrito Potato,1.0,,Breakfast,Total Carbohydrate (g),116.0,269.77
3,,,,,Dietary Fiber (g),14.0,155.56
4,,,,,Total Fat (g),51.0,164.52
5,,,,,Saturated Fat (g),12.0,150.00
6,,,,,Trans Fat (g),0.0,0.00
7,,,,,Cholesterol (mg),125.0,125.00
8,,,,,Sodium (mg),2170.0,283.29
9,,,,,Sugars (g),10.0,333.33


# Model 5: Minimizing Cost, Mix-Integer Programming
### Decision Variables
* $x_i$: The number of menu items $i$ to include in the diet; $x_1,..,n$
* $y_i=1$ if food item $i$ from menu section $z$ is served, and 0 otherwise: $i \in F, z \in S$
i∈F,z∈S$
* $k$ =The total number of calories

### Objective
* Minimize costs $\sum_{i \in F} c_i * x_i$.


### Nutrional Constraints
* $ \sum_{i \in F} n_{i,j} * x_i \geq m_j,  j \in N$

### Upper bound on items 
* $x_i \leq 1  \forall i \in F$

### Section Constraints
* $\sum_{ i \in F, z_{i} = z} x_i \geq 1 * y_z \forall z \in S$
* $\sum_{z \in S} y_z \geq 3$

### Non-Negativity Constraint
* $x_i \geq 0, i=1,\ldots,n$ (non-negativity, and integer constraint)
* $y_z \in {0, 1} \forall z \in S$ (binary)  

In [13]:
m_male5_MIP = gp.Model("TBELL_MODEL: EOM-MIP")

m_male5_MIP.Params.LogToConsole = 1 # Noisy output



foods = [m_male5_MIP.addVar(obj=food_data['price'][i], vtype="I",ub=1, name=food_data['item_name'][i]) for i in range(len(food_data))]

cal = m_male5_MIP.addVar(vtype = "C", name = 'Calories', ub = 2400)

sections = food_data['menu_section'].unique()

section_served = {section: m_male5_MIP.addVar(vtype="B", name=f"serve_{section}") for section in sections}


max_protein = (cal*0.35)/4
min_protein = (cal*0.10)/4

max_carbs = (cal*0.65)/4
min_carbs = (cal*0.45)/4

min_totFat = (cal*0.2)/9
max_totFat = (cal*0.35)/9

max_statFat = (cal*0.10)/9

m_male5_MIP.setObjective(gp.quicksum(foods[i]*price[i] for i in range(len(food_data))), GRB.MINIMIZE)




#### Constraints
# Macro Nutrients  
m_male5_MIP.addConstr(gp.quicksum(calories[i]*foods[i] for i in range(len(food_data))) >= cal/3, name="calories")
m_male5_MIP.addConstr(gp.quicksum(protein[i]*foods[i] for i in range(len(food_data))) >= 56/3,name="protein")
m_male5_MIP.addConstr(gp.quicksum(protein[i]*foods[i] for i in range(len(food_data))) <= max_protein/3,name="protein_MAX")
m_male5_MIP.addConstr(gp.quicksum(protein[i]*foods[i] for i in range(len(food_data))) >= min_protein/3,name="protein_MIN")
m_male5_MIP.addConstr(gp.quicksum(totalCarbohydratets[i]*foods[i] for i in range(len(food_data))) >= 130/3, name="totalCarbohydratets")
m_male5_MIP.addConstr(gp.quicksum(totalCarbohydratets[i]*foods[i] for i in range(len(food_data))) <= max_carbs/3, name="totalCarbohydratets_MAX")
m_male5_MIP.addConstr(gp.quicksum(totalCarbohydratets[i]*foods[i] for i in range(len(food_data))) >= min_carbs/3, name="totalCarbohydratets_MIN")
m_male5_MIP.addConstr(gp.quicksum(dietaryFiber[i]*foods[i] for i in range(len(food_data))) >= 34/3, name="dietaryFiber")
m_male5_MIP.addConstr(gp.quicksum(totfat[i]*foods[i] for i in range(len(food_data))) >= max_totFat/3, name="totFat")
m_male5_MIP.addConstr(gp.quicksum(totfat[i]*foods[i] for i in range(len(food_data))) <= max_totFat/3, name="totFat_MAX")
m_male5_MIP.addConstr(gp.quicksum(totfat[i]*foods[i] for i in range(len(food_data))) >= min_totFat/3, name="totFat_MIN")
m_male5_MIP.addConstr(gp.quicksum(statFat[i]*foods[i] for i in range(len(food_data))) >= max_statFat/3, name="statFat")
m_male5_MIP.addConstr(gp.quicksum(transFat[i]*foods[i] for i in range(len(food_data))) <= 0/3, name="transFat")
m_male5_MIP.addConstr(gp.quicksum(cholesterol[i]*foods[i] for i in range(len(food_data))) >= 300/3, name="cholesterol")
m_male5_MIP.addConstr(gp.quicksum(sodium[i]*foods[i] for i in range(len(food_data))) >= 2300/3, name="sodium")
m_male5_MIP.addConstr(gp.quicksum(sugars[i]*foods[i] for i in range(len(food_data)))>=10/3, name="sugars")
# Micro Nutrients
m_male5_MIP.addConstr(gp.quicksum(Potassium[i]*foods[i] for i in range(len(food_data))) >= 3400/3, name="Potassium")
m_male5_MIP.addConstr(gp.quicksum(iron[i]*foods[i] for i in range(len(food_data))) >= 8/3, name="iron")
m_male5_MIP.addConstr(gp.quicksum(Calcium[i]*foods[i] for i in range(len(food_data))) >= 1000/3, name="Calcium")


# Constraint for each section: if y_i = 1, then at least one item from that section is served
for section in sections:
    m_male5_MIP.addConstr(gp.quicksum(foods[i] for i in range(len(food_data)) if section_name[i] == section) >= section_served[section], f"serve_at_least_one_{section}")

# Constraint to ensure a minimum number of sections are served
min_sections_served = 3
m_male5_MIP.addConstr(gp.quicksum(section_served[section] for section in sections) >= min_sections_served, "min_sections_served")


m_male5_MIP.ModelSense = 1 #Minimization problem
m_male5_MIP.optimize()


Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M2 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 29 rows, 73 columns and 1167 nonzeros
Model fingerprint: 0x60fc721c
Variable types: 1 continuous, 72 integer (9 binary)
Coefficient statistics:
  Matrix range     [4e-03, 2e+03]
  Objective range  [1e+00, 8e+00]
  Bounds range     [1e+00, 2e+03]
  RHS range        [3e+00, 1e+03]
Presolve removed 9 rows and 30 columns
Presolve time: 0.00s
Presolved: 20 rows, 43 columns, 493 nonzeros
Variable types: 0 continuous, 43 integer (42 binary)

Root relaxation: objective 9.773029e+00, 21 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    9.77303    0    5          -    9.77303      -     -    0s

Cutting planes:
  Cover: 2
  MIR: 2
  StrongCG: 1
  RLT: 

In [18]:
if m_male5_MIP.status == gp.GRB.OPTIMAL:
    print(f'Optimal combination of foods for one meal costs ${round(m_male5_MIP.objVal, 2)}:')
    
    df5 = output_to_table(m_male5_MIP, foods, food_data)

    df5

else:
    print("Model is infeasible")

Model is infeasible
